# Week 1 study case.
## Study requirements.
USE SEAL!!!!!
## Participants.
1. Data providers - individuals (alba-saeng).
2. Client (The government).
3. Service provider (A surveying company).
## Scenario.
0.  The client has to decide
    *  The level of security (Probably bound be the law).
    *  The number of samples (Probably bound by the price of the service).
1.  The client asks for a service to calculate the __average__ age of data providers to the service provider.
2.  The service provider hands over the __*key generating program*__ to generate
    *  a public key.
    *  a set of evaluation keys.
    *  an appropriate homomorphic encryption parameters. (encryption policy)
3.  The client generates the necessary requirements, using the  __*key generating program*__, together with the desired number of samples, and remits the public key, the evaluation key and the number of samples to the service provider. The client holds the private key and does not reveal it.
4.  The service provider calls the data providers and distributes the __*encrypting program*__ that contains embedded public keys (the service provider holds the list of data providers).
5.  Data providers encrypt their ages using the __*encrypting program*__ and submit the encrypted data to the service provider.
6.  The service provider calculates the __average__ age in encrypted domain, then return the result to the client.
7.  The client decrypts the result with the private key the party holds to itself.

In [1]:
import numpy as np
import tenseal as ts
import time

In [2]:
# parameters
# FRACTIONAL_BITS = 15
# Client
security_level = {
    'scheme_type': ts.SCHEME_TYPE.CKKS,
    'poly_modulus_degree': 4096, # 4096
    'plain_modulus': 1032193, #1032193
    'coeff_modulus_bit_sizes': [40, 20, 40]
}
# create TenSEALContext
ctx_eval = ts.context(security_level['scheme_type'], security_level['poly_modulus_degree'], -1, security_level['coeff_modulus_bit_sizes'])
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()

In [22]:
n_count = 1000
plain_vector = np.random.randint(10, 80, size=n_count).tolist()

In [4]:
t_start = time.time()
enc_plain_vector = [ts.ckks_vector(ctx_eval, [x]) for x in plain_vector]
t_end = time.time()
print(f"Encrypting data took {(t_end - t_start):.4f} seconds")

Encrypting data took 0.9641 seconds


In [5]:
def encrypted_evaluation(enc_data):
    enc_result = 0
    count = len(enc_data)
    for enc_x in enc_data:
        if enc_result == 0:
            enc_result = enc_x
        else:
            enc_result = enc_result + enc_x
    enc_result *= (1/count)
    return enc_result

In [6]:
t_start = time.time()
encrypted_average = encrypted_evaluation(enc_plain_vector)

print(encrypted_average.decrypt(), np.average(plain_vector))
t_end = time.time()
print(f"Encryption average took {(t_end - t_start):.4f} seconds")

[44.861437627548966] 44.144
Encryption average took 0.0172 seconds


In [23]:
t_start = time.time()
enc_vec_batch = ts.ckks_vector(ctx_eval, plain_vector)
enc_vec_count = ts.ckks_vector(ctx_eval, [(1/n_count)]*n_count)
t_end = time.time()
print(f"Encrypting batch data took {(t_end - t_start):.4f} seconds", len(plain_vector))

Encrypting batch data took 0.0069 seconds 1000


In [8]:
def encrypted_evaluation_batch(enc_data, dividend):
    return enc_data * dividend

In [17]:
t_start = time.time()
encrypted_division = encrypted_evaluation_batch(enc_vec_batch, n_count)

print(np.sum(encrypted_division.decrypt()), np.average(plain_vector))
t_end = time.time()
print(f"Encryption average took {(t_end - t_start):.4f} seconds")

4525205815.737571 44.54504
Encryption average took 0.0553 seconds


In [15]:
# experiment start

import pandas as pd

experiment = [
    {
        'n_count': 1000,
        'enc_avg': 0,
        'enc_avg_batch': 0,
        'enc_time': 0,
        'enc_avg_time': 0,
        'avg': 0
    },
    
    {
        'n_count': 10000,
        'enc_avg': 0,
        'enc_avg_batch': 0,
        'avg': 0
    },

    {
        'n_count': 100000,
        'enc_avg': 0,
        'enc_avg_batch': 0,
        'avg': 0
    }
]


In [18]:
for i, exp in enumerate(experiment):
    print(f"Start {i + 1}th Experiment !!")

    # data from data provider :
    plain_vector = np.random.randint(10, 80, size=exp['n_count']).tolist()

    # encrypt data
    t_start = time.time()
    enc_vec = [ts.ckks_vector(ctx_eval, [x]) for x in plain_vector]
    t_end = time.time()

    experiment[i]['enc_time'] = t_end - t_start
    print(f"Encrypting data took {(t_end - t_start):.4f} seconds")

    # encrypt data batch
    t_start = time.time()
    enc_vec_batch = ts.ckks_vector(ctx_eval, plain_vector)
    enc_vec_count = ts.ckks_vector(ctx_eval, [(1/exp['n_count'])]*exp['n_count'])
    t_end = time.time()
    experiment[i]['enc_batch_time'] = t_end - t_start
    print(f"Encrypting batch data took {(t_end - t_start):.4f} seconds")

    # encrypted evaluation
    t_start = time.time()
    encrypted_average = encrypted_evaluation(enc_vec)
    avg_result = encrypted_average.decrypt()
    t_end = time.time()

    experiment[i]['enc_avg'] = avg_result
    experiment[i]['avg'] = np.average(plain_vector)

    experiment[i]['enc_avg_time'] = t_end - t_start
    print(f"Encryption average took {(t_end - t_start):.4f} seconds")

    # encrypted batch evaluation
    t_start = time.time()
    encrypted_division = encrypted_evaluation_batch(enc_vec_batch, n_count)
    avg_result = np.sum(encrypted_division.decrypt())
    t_end = time.time()

    experiment[i]['enc_avg_batch'] = np.sum(avg_result)
    experiment[i]['enc_avg_batch_time'] = t_end - t_start
    print(f"Encryption average batch took {(t_end - t_start):.4f} seconds")





Start 1th Experiment !!
Encrypting data took 0.9371 seconds
Encrypting batch data took 0.0023 seconds
Encryption average took 0.0172 seconds
Encryption average took 0.0004 seconds
Start 2th Experiment !!
Encrypting data took 9.1638 seconds
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
Encrypting batch data took 0.0072 seconds
Encryption average took 0.1525 seconds
Encryption average took 0.0019 seconds
Start 3th Experiment !!
Encrypting data took 91.5855 seconds
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
Encrypting batch data took 0.0645 seconds
Encryption average took 1.3902 seconds
Encryption average took 0.0191 seconds


In [19]:
exp_df = pd.DataFrame(experiment)
print(exp_df)

   n_count               enc_avg  enc_avg_batch   enc_time  enc_avg_time  \
0     1000  [43.755069072307876]   4.373533e+07   0.937114      0.017159   
1    10000   [45.53474691814647]   4.547499e+08   9.163823      0.152519   
2   100000   [43.05938914993923]   4.514986e+09  91.585541      1.390153   

        avg  enc_batch_time  enc_avg_batch_time  
0  43.05200        0.002251            0.000392  
1  44.76450        0.007228            0.001923  
2  44.44444        0.064510            0.019071  
